# import

In [1]:
# IMPORTS
from distutils.command.config import config
import pickle5 as pickle

# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import wandb
import os
from os.path import exists

# Evaluation
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_auc_score, \
    accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

import wandb
import glob

# data
import numpy as np
import matplotlib.pyplot as plt

#torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch.utils.data import WeightedRandomSampler

# Load data

In [2]:
# CURR_PATH = os.getcwd()
CURR_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM'
DATA_PATH = '/home/svetlanamaslenkova/Documents/data/physionet.org/files/mimiciv/2.0/'
DF_PATH = CURR_PATH +'/dataframes_2/'
# ED_PATH = DATA_PATH+'physionet.org/files/mimic-iv-ed/1.0/ed/'
PKL_PATH = CURR_PATH+'/pickles_2/'
DF_PATH = CURR_PATH+'/dataframes_2/'
ICU_PATH = DATA_PATH + 'icu/'
HOSP_PATH = DATA_PATH + 'hosp/'
XGB_DATA_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/xgb/dataframes/'

In [193]:
import pandas as pd

data_patients = pd.read_csv(HOSP_PATH+'patients.csv')

data_patients.columns = data_patients.columns.str.lower()

data_patients.head(5)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [3]:
LABELS_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM/pickles_2/aki_stage_labels.pkl'

with open(LABELS_PATH, 'rb') as f:
    aki_stage_labels = pickle.load(f)

In [15]:
with open(PKL_PATH + 'multihot_meds_icu_day_0.pkl', 'rb') as f:
    data_medications = pickle.load(f)
print(data_medications.stay_id.unique().shape[0])
data_medications.head()

70205


,subject_id,hadm_id,stay_id,med_ACD-A Citrate (1000ml),med_ACD-A Citrate (500ml),med_Abciximab (Reopro),med_Acetaminophen-IV,med_Acetylcysteine,med_Adenosine,med_Alteplase (TPA),...,med_Ranitidine (Prophylaxis),med_Rocuronium,med_Sodium Bicarbonate 8.4%,med_Thiamine,med_Tirofiban (Aggrastat),med_Treprostinil (Remodulin),med_Vasopressin,med_Vecuronium,med_Verapamil,med_Vitamin K (Phytonadione)
0,10000980,26913865,39765666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,225911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001217,24597018,37067082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001217,27703517,34592300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001725,25563031,31205490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001884,26184834,37510196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
with open(PKL_PATH + 'multihot_labs_icu_day_0.pkl', 'rb') as f:
    data_labs = pickle.load(f)
data_labs['stay_id'] = data_labs['stay_id'].astype('int64')
print(data_labs.stay_id.unique().shape[0])
data_labs.head()

75393


,subject_id,hadm_id,stay_id,icu_day_id,first_ALT,first_AST,first_AT (Antithrombin funct),first_Absolute Count - Basos,first_Absolute Count - Eos,first_Absolute Count - Lymphs,...,last_Triglyceride,last_Troponin-T,last_UF Heparin (Anti-Xa),last_Uric Acid,last_Vancomycin (Peak),last_Vancomycin (Random),last_Vancomycin (Trough),last_Venous CO2 Pressure,last_Venous O2 Pressure,last_WBC
0,10000032,29079034,39553978,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000980,26913865,39765666,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001217,24597018,37067082,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
3,10001217,27703517,34592300,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4
4,10001725,25563031,31205490,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.1


In [97]:
with open(PKL_PATH + 'multihot_vitals_icu_day_0.pkl', 'rb') as f:
    data_vitals = pickle.load(f)
data_vitals['stay_id'] = data_vitals['stay_id'].astype('int64')
print(data_vitals.stay_id.unique().shape[0])
data_vitals.head()

36841


,subject_id,hadm_id,stay_id,icu_day_id,first_ABPd,first_ABPm,first_ABPs,first_ART BP Diastolic,first_ART BP Mean,first_ART BP Systolic,...,max_Temperature C,max_Temperature F,max_obpd lying,max_obps lying,max_obps sitting,max_obps standing,max_ohr lying,max_ohr sitting,max_ohr standing,max_rhr aar
0,10001884,26184834,37510196,0,NaN,NaN,NaN,NaN,NaN,NaN,...,35.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002155,23822395,33685454,0,59.0,82.0,115.0,NaN,NaN,NaN,...,NaN,95.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002155,28994087,31090461,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10002348,22725460,32610785,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10002428,20321825,34807493,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
with open(XGB_DATA_PATH + 'data_output_xgb.pkl', 'rb') as f:
    data_outputs = pickle.load(f)
data_outputs['stay_id'] = data_outputs['stay_id'].astype('int64')
data_outputs = data_outputs[(data_outputs.icu_day_id==0)&(data_outputs.stay_id.isin(stays))]
print(data_outputs.stay_id.unique().shape[0])
data_outputs.head()

14077


abbreviation,subject_id,hadm_id,stay_id,icu_day_id,output
26,10002155,28994087,31090461,0,3260.0
33,10002348,22725460,32610785,0,325.0
42,10002428,23473524,35479615,0,2255.0
53,10002428,28662225,33987268,0,805.0
58,10002428,28662225,38875437,0,1630.0


In [111]:
with open(XGB_DATA_PATH + 'data_medications_xgb.pkl', 'rb') as f:
    data_medications = pickle.load(f)
data_medications['stay_id'] = data_medications['stay_id'].astype('int64')
data_medications = data_medications[(data_medications.stay_id.isin(stays))]
print(data_medications.stay_id.unique().shape[0])
data_medications.head()

14282


,stay_id,M01,B01,C03,C02
2,31090461,0,1,0,0
3,32610785,0,0,0,0
5,35479615,0,1,0,0
6,33987268,0,1,0,0
7,38875437,0,0,0,0


In [224]:
with open(XGB_DATA_PATH + 'data_demographics_xgb.pkl', 'rb') as f:
    data_demographics = pickle.load(f)
data_demographics = data_demographics[data_demographics.stay_id.isin(stays)]
print(data_demographics.stay_id.unique().shape[0])
data_demographics.head()

14281


,subject_id,hadm_id,stay_id,race,gender,age,BMI,Height,Weight,diastolic_bp,systolic_bp
110,10002155,28994087,31090461,1,1,82,21.4,60.0,109.4,51.0,99.0
121,10002348,22725460,32610785,1,1,77,NaN,NaN,NaN,NaN,NaN
123,10002428,23473524,35479615,1,1,81,20.0,59.0,99.0,78.0,131.0
127,10002428,28662225,33987268,1,1,81,20.0,59.0,99.0,78.0,131.0
128,10002428,28662225,33987268,1,1,81,20.0,59.0,99.0,78.0,131.0


In [130]:
with open(XGB_DATA_PATH + 'data_xgb.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset.columns = [col.replace(' ', '_') for col in dataset.columns]
dataset.columns = dataset.columns.str.lower()
print(dataset.shape)
dataset.head()

(33805, 65)


,subject_id,hadm_id,stay_id,bmi,height,weight,diastolic_bp,systolic_bp,min_sodium_(serum),max_chloride_(serum),...,min_nbpm,max_nbpd,min_nbpd,output,race,gender,age,aki_1,aki_2,aki_3
0,10002155,28994087,31090461,21.4,60.0,109.4,51.0,99.0,136.0,106.0,...,76.0,25.0,64.0,3260.0,1,1,82,0,0,0
1,10002428,20321825,34807493,20.0,59.0,99.0,78.0,131.0,133.0,93.0,...,112.0,32.0,101.0,888.0,1,1,81,0,0,0
2,10002428,23473524,35479615,20.0,59.0,99.0,78.0,131.0,135.0,107.0,...,NaN,NaN,NaN,2255.0,1,1,81,0,0,0
3,10002428,28662225,33987268,20.0,59.0,99.0,78.0,131.0,133.0,98.0,...,92.0,19.0,90.0,805.0,1,1,81,1,1,0
5,10002428,28662225,38875437,20.0,59.0,99.0,78.0,131.0,144.0,110.0,...,127.0,30.0,116.0,1630.0,1,1,81,1,1,0


In [155]:
dataset_demo = dataset[['subject_id',	'hadm_id',	'stay_id', 'bmi',	'height',	'weight',	'diastolic_bp',	'systolic_bp', 'race',	'gender',	'age']]
dataset_demo['gender'] = dataset_demo['gender'].astype(int)
dataset_demo['age'] = dataset_demo['age'].astype(int)

/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
dataset_demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33805 entries, 0 to 41469
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subject_id    33805 non-null  int64  
 1   hadm_id       33805 non-null  int64  
 2   stay_id       33805 non-null  int64  
 3   bmi           14263 non-null  float64
 4   height        12108 non-null  float64
 5   weight        15510 non-null  float64
 6   diastolic_bp  13847 non-null  float64
 7   systolic_bp   13847 non-null  float64
 8   race          33805 non-null  int64  
 9   gender        33805 non-null  object 
 10  age           33805 non-null  object 
dtypes: float64(5), int64(4), object(2)
memory usage: 3.1+ MB


In [24]:
with open(DF_PATH + 'pid_train_dataset_icu.pkl', 'rb') as f:
    pid_train_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_test_dataset_icu.pkl', 'rb') as f:
    pid_test_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_val_dataset_icu.pkl', 'rb') as f:
    pid_val_dataset_icu = pickle.load(f)

In [161]:
stays = [*list(pid_train_dataset_icu.stay_id.unique()), *list(pid_test_dataset_icu.stay_id.unique()),*list(pid_val_dataset_icu.stay_id.unique())]
len(stays)

14282

In [486]:
aki_stage_labels = aki_stage_labels[aki_stage_labels.stay_id.isin(stays)&(aki_stage_labels.icu_day_id==1)]
aki_stage_labels.stay_id.unique().shape

(13128,)

In [163]:
def difference(lst1, lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    #return lst3
    return lst3

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    #return lst3
    return lst3

In [165]:
len(intersection(stays, aki_stage_labels.stay_id.unique()))

13128

In [487]:
pid_train_dataset_icu = pid_train_dataset_icu[pid_train_dataset_icu.stay_id.isin(intersection(stays, aki_stage_labels.stay_id.unique()))]
pid_test_dataset_icu = pid_test_dataset_icu[pid_test_dataset_icu.stay_id.isin(intersection(stays, aki_stage_labels.stay_id.unique()))]
pid_val_dataset_icu = pid_val_dataset_icu[pid_val_dataset_icu.stay_id.isin(intersection(stays, aki_stage_labels.stay_id.unique()))]

In [488]:
train_stays = pid_train_dataset_icu.stay_id.unique()
test_stays = pid_test_dataset_icu.stay_id.unique()
val_stays = pid_val_dataset_icu.stay_id.unique()

In [489]:
print(len(train_stays))
print(len(test_stays))
print(len(val_stays))

10487
1328
1313


# Preproccesing

## Filter Missing

### Labs

In [98]:
data_labs = data_labs[data_labs.stay_id.isin(stays)]
print(data_labs.stay_id.unique().shape[0])

14246


In [56]:
for k,v in data_labs.isna().sum().sort_values(ascending=False).iteritems():
    print(k,v)

last_ScvO2 Central Venous O2% Sat 14246
first_Vancomycin (Peak) 14246
last_Vancomycin (Peak) 14246
first_ScvO2 Central Venous O2% Sat 14246
first_Tobramycin (Peak) 14244
last_Tobramycin (Peak) 14244
last_Gentamicin (Peak) 14243
first_Gentamicin (Peak) 14243
last_Phenytoin (Free) 14237
first_Phenytoin (Free) 14237
last_AT (Antithrombin funct) 14236
first_AT (Antithrombin funct) 14236
last_Tobramycin (Trough) 14235
first_Tobramycin (Trough) 14235
last_UF Heparin (Anti-Xa) 14234
first_UF Heparin (Anti-Xa) 14234
first_Tobramycin (Random) 14233
last_Tobramycin (Random) 14233
first_Gentamicin (Random) 14231
last_Gentamicin (Random) 14231
last_Gentamicin (Trough) 14231
first_Gentamicin (Trough) 14231
last_Thrombin 14224
first_Thrombin 14224
first_Total Granulocyte Count (TGC) 14215
last_Total Granulocyte Count (TGC) 14215
first_Creatinine (whole blood) 14212
last_Creatinine (whole blood) 14212
last_Phenobarbital 14202
first_Phenobarbital 14202
last_Absolute Neutrophil Count 14201
first_Absolu

In [99]:
cols_to_drop_labs = data_labs.isna().sum().sort_values(ascending=False).index[:119]
data_labs = data_labs.drop(columns=cols_to_drop_labs)

In [144]:
data_labs.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'first_ALT',
       'first_AST', 'first_Albumin', 'first_Alkaline Phosphate',
       'first_Anion gap', 'first_Arterial Base Excess',
       'first_Arterial CO2 Pressure', 'first_Arterial O2 pressure',
       'first_BUN', 'first_CK (CPK)', 'first_CK-MB',
       'first_Calcium non-ionized', 'first_Chloride (serum)',
       'first_Creatinine (serum)', 'first_Differential-Basos',
       'first_Differential-Eos', 'first_Differential-Lymphs',
       'first_Differential-Monos', 'first_Differential-Neuts',
       'first_Glucose (serum)', 'first_Glucose (whole blood)',
       'first_Glucose finger stick (range 70-100)', 'first_HCO3 (serum)',
       'first_Hematocrit (serum)', 'first_Hemoglobin', 'first_INR',
       'first_Ionized Calcium', 'first_LDH', 'first_Lactic Acid',
       'first_Magnesium', 'first_PH (Arterial)', 'first_PH (dipstick)',
       'first_PTT', 'first_Phosphorous', 'first_Platelet Count',
       'first_Potassium (serum

In [314]:
data_labs_columns = list(data_labs.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id']).columns.str.lower())

### Vitals

In [101]:
data_vitals = data_vitals[data_vitals.stay_id.isin(stays)]
print(data_vitals.stay_id.unique().shape[0])

14256


In [102]:
for k,v in data_vitals.isna().sum().sort_values(ascending=False).iteritems():
    print(k,v)

max_GlucoseScore_ApacheIV 14256
last_Glucose_ApacheIV 14256
max_Glucose_ApacheIV 14256
first_Glucose_ApacheIV 14256
first_GlucoseScore_ApacheIV 14256
median_Glucose_ApacheIV 14256
median_GlucoseScore_ApacheIV 14256
min_GlucoseScore_ApacheIV 14256
min_Glucose_ApacheIV 14256
last_GlucoseScore_ApacheIV 14256
last_Rest HR - Aerobic Capacity 14253
first_Rest HR - Aerobic Capacity 14253
max_Rest HR - Aerobic Capacity 14253
median_Rest HR - Aerobic Capacity 14253
min_Rest HR - Aerobic Capacity 14253
median_rhr aar 14250
first_rhr aar 14250
max_rhr aar 14250
min_rhr aar 14250
last_rhr aar 14250
min_ohr standing 14230
min_obps standing 14230
median_ohr standing 14230
median_obps standing 14230
max_obps standing 14230
last_ohr standing 14230
last_obps standing 14230
max_ohr standing 14230
first_ohr standing 14230
first_obps standing 14230
median_ohr lying 14227
first_ohr sitting 14227
median_ohr sitting 14227
first_ohr lying 14227
last_ohr sitting 14227
min_ohr lying 14227
max_ohr sitting 14227


In [103]:
cols_to_drop_vitals = data_vitals.isna().sum().sort_values(ascending=False).index[:130]
data_vitals = data_vitals.drop(columns=cols_to_drop_vitals)

In [142]:
data_vitals.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'first_HR',
       'first_HR Alarm - High', 'first_HR Alarm - Low', 'first_NBPd',
       'first_NBPm', 'first_NBPs', 'first_Temperature F', 'last_HR',
       'last_HR Alarm - High', 'last_HR Alarm - Low', 'last_NBPd', 'last_NBPm',
       'last_NBPs', 'last_Temperature F', 'median_HR',
       'median_HR Alarm - High', 'median_HR Alarm - Low', 'median_NBPd',
       'median_NBPm', 'median_NBPs', 'median_Temperature F', 'min_HR',
       'min_HR Alarm - High', 'min_HR Alarm - Low', 'min_NBPd', 'min_NBPm',
       'min_NBPs', 'min_Temperature F', 'max_HR', 'max_HR Alarm - High',
       'max_HR Alarm - Low', 'max_NBPd', 'max_NBPm', 'max_NBPs',
       'max_Temperature F'],
      dtype='object')

In [315]:
data_vitals_columns = list(data_vitals.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id']).columns.str.lower())

### Medications

In [63]:
data_medications = data_medications[data_medications.stay_id.isin(stays)]
print(data_medications.stay_id.unique().shape[0])

14282


In [70]:
data_medications

,stay_id,M01,B01,C03,C02
2,31090461,0,1,0,0
3,32610785,0,0,0,0
5,35479615,0,1,0,0
6,33987268,0,1,0,0
7,38875437,0,0,0,0
...,...,...,...,...,...
36900,37403074,0,0,0,0
36901,31417783,0,1,1,1
36903,31144045,0,1,1,0
36906,30143796,0,0,0,0


In [316]:
data_medications_columns = list(data_medications.drop(columns='stay_id').columns.str.lower())

### Demographics

In [118]:
data_labs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14246 entries, 9 to 75387
Data columns (total 93 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   subject_id                                 14246 non-null  uint64 
 1   hadm_id                                    14246 non-null  uint64 
 2   stay_id                                    14246 non-null  int64  
 3   icu_day_id                                 14246 non-null  int64  
 4   first_ALT                                  5946 non-null   float64
 5   first_AST                                  5943 non-null   float64
 6   first_Albumin                              3956 non-null   float64
 7   first_Alkaline Phosphate                   5829 non-null   float64
 8   first_Anion gap                            14065 non-null  float64
 9   first_Arterial Base Excess                 6694 non-null   float64
 10  first_Arterial CO2 Pre

In [137]:
data_demographics = dataset_demo[dataset_demo.stay_id.isin(stays)].drop_duplicates()

In [188]:
data_demographics.isna().sum()

subject_id          0
hadm_id             0
stay_id             0
bmi              9667
height          10108
weight           9399
diastolic_bp     9432
systolic_bp      9432
race                0
gender              0
age                 0
dtype: int64

In [317]:
dataset_demo_cols = list(dataset_demo.drop(columns=['subject_id', 'hadm_id', 'stay_id']).columns.str.lower())

### Merge data

In [392]:
data = data_demographics.merge(data_labs, on='stay_id', how='inner').merge(data_vitals, on='stay_id', how='left').merge(data_outputs, on=['stay_id', 'hadm_id'],how='left').merge(data_medications, on='stay_id', how='left')

In [365]:
data.shape

(18076, 148)

In [288]:
data = data.iloc[:, 2:]

In [393]:
data = data.rename(columns={'subject_id_x':'subject_id', })

In [394]:
data = data[['stay_id',
 'subject_id',
 'hadm_id',
 'icu_day_id',
 'race',
 'gender',
 'age',
 'BMI',
 'Height',
 'Weight',
 'diastolic_bp',
 'systolic_bp',
 'first_ALT',
 'first_AST',
 'first_Albumin',
 'first_Alkaline Phosphate',
 'first_Anion gap',
 'first_Arterial Base Excess',
 'first_Arterial CO2 Pressure',
 'first_Arterial O2 pressure',
 'first_BUN',
 'first_CK (CPK)',
 'first_CK-MB',
 'first_Calcium non-ionized',
 'first_Chloride (serum)',
 'first_Creatinine (serum)',
 'first_Differential-Basos',
 'first_Differential-Eos',
 'first_Differential-Lymphs',
 'first_Differential-Monos',
 'first_Differential-Neuts',
 'first_Glucose (serum)',
 'first_Glucose (whole blood)',
 'first_Glucose finger stick (range 70-100)',
 'first_HCO3 (serum)',
 'first_Hematocrit (serum)',
 'first_Hemoglobin',
 'first_INR',
 'first_Ionized Calcium',
 'first_LDH',
 'first_Lactic Acid',
 'first_Magnesium',
 'first_PH (Arterial)',
 'first_PH (dipstick)',
 'first_PTT',
 'first_Phosphorous',
 'first_Platelet Count',
 'first_Potassium (serum)',
 'first_Potassium (whole blood)',
 'first_Prothrombin time',
 'first_Sodium (serum)',
 'first_Specific Gravity (urine)',
 'first_TCO2 (calc) Arterial',
 'first_Total Bilirubin',
 'first_Troponin-T',
 'first_WBC',
 'last_ALT',
 'last_AST',
 'last_Albumin',
 'last_Alkaline Phosphate',
 'last_Anion gap',
 'last_Arterial Base Excess',
 'last_Arterial CO2 Pressure',
 'last_Arterial O2 pressure',
 'last_BUN',
 'last_CK (CPK)',
 'last_CK-MB',
 'last_Calcium non-ionized',
 'last_Chloride (serum)',
 'last_Creatinine (serum)',
 'last_Differential-Basos',
 'last_Differential-Eos',
 'last_Differential-Lymphs',
 'last_Differential-Monos',
 'last_Differential-Neuts',
 'last_Glucose (serum)',
 'last_Glucose (whole blood)',
 'last_Glucose finger stick (range 70-100)',
 'last_HCO3 (serum)',
 'last_Hematocrit (serum)',
 'last_Hemoglobin',
 'last_INR',
 'last_Ionized Calcium',
 'last_LDH',
 'last_Lactic Acid',
 'last_Magnesium',
 'last_PH (Arterial)',
 'last_PH (dipstick)',
 'last_PTT',
 'last_Phosphorous',
 'last_Platelet Count',
 'last_Potassium (serum)',
 'last_Potassium (whole blood)',
 'last_Prothrombin time',
 'last_Sodium (serum)',
 'last_Sodium (whole blood)',
 'last_Specific Gravity (urine)',
 'last_TCO2 (calc) Arterial',
 'last_Total Bilirubin',
 'last_Troponin-T',
 'last_WBC',
 'hadm_id',
 'first_HR',
 'first_HR Alarm - High',
 'first_HR Alarm - Low',
 'first_NBPd',
 'first_NBPm',
 'first_NBPs',
 'first_Temperature F',
 'last_HR',
 'last_HR Alarm - High',
 'last_HR Alarm - Low',
 'last_NBPd',
 'last_NBPm',
 'last_NBPs',
 'last_Temperature F',
 'median_HR',
 'median_HR Alarm - High',
 'median_HR Alarm - Low',
 'median_NBPd',
 'median_NBPm',
 'median_NBPs',
 'median_Temperature F',
 'min_HR',
 'min_HR Alarm - High',
 'min_HR Alarm - Low',
 'min_NBPd',
 'min_NBPm',
 'min_NBPs',
 'min_Temperature F',
 'max_HR',
 'max_HR Alarm - High',
 'max_HR Alarm - Low',
 'max_NBPd',
 'max_NBPm',
 'max_NBPs',
 'max_Temperature F',
 'icu_day_id',
 'output',
 'M01',
 'B01',
 'C03',
 'C02']]

In [395]:
data.shape

(18076, 144)

In [369]:
data.columns.to_list()

['stay_id',
 'subject_id',
 'subject_id',
 'hadm_id',
 'icu_day_id',
 'race',
 'gender',
 'age',
 'BMI',
 'Height',
 'Weight',
 'diastolic_bp',
 'systolic_bp',
 'first_ALT',
 'first_AST',
 'first_Albumin',
 'first_Alkaline Phosphate',
 'first_Anion gap',
 'first_Arterial Base Excess',
 'first_Arterial CO2 Pressure',
 'first_Arterial O2 pressure',
 'first_BUN',
 'first_CK (CPK)',
 'first_CK-MB',
 'first_Calcium non-ionized',
 'first_Chloride (serum)',
 'first_Creatinine (serum)',
 'first_Differential-Basos',
 'first_Differential-Eos',
 'first_Differential-Lymphs',
 'first_Differential-Monos',
 'first_Differential-Neuts',
 'first_Glucose (serum)',
 'first_Glucose (whole blood)',
 'first_Glucose finger stick (range 70-100)',
 'first_HCO3 (serum)',
 'first_Hematocrit (serum)',
 'first_Hemoglobin',
 'first_INR',
 'first_Ionized Calcium',
 'first_LDH',
 'first_Lactic Acid',
 'first_Magnesium',
 'first_PH (Arterial)',
 'first_PH (dipstick)',
 'first_PTT',
 'first_Phosphorous',
 'first_Platele

In [396]:
temp = data['subject_id'].iloc[:, 0]
data = data.drop(columns='subject_id')
data['subject_id'] = temp

temp = data['hadm_id'].iloc[:, 0]
data = data.drop(columns='hadm_id')
data['hadm_id'] = temp

temp = data['icu_day_id'].iloc[:, 0]
data = data.drop(columns='icu_day_id')
data['icu_day_id'] = temp

In [398]:
data.columns = data.columns.str.lower()

In [399]:
data = data[['stay_id',
    'subject_id',
    'hadm_id',
    'icu_day_id',
    'race',
    'gender',
    'age',
    'bmi',
    'height',
    'weight',
    'diastolic_bp',
    'systolic_bp',
    'first_alt',
    'first_ast',
    'first_albumin',
    'first_alkaline phosphate',
    'first_anion gap',
    'first_arterial base excess',
    'first_arterial co2 pressure',
    'first_arterial o2 pressure',
    'first_bun',
    'first_ck (cpk)',
    'first_ck-mb',
    'first_calcium non-ionized',
    'first_chloride (serum)',
    'first_creatinine (serum)',
    'first_differential-basos',
    'first_differential-eos',
    'first_differential-lymphs',
    'first_differential-monos',
    'first_differential-neuts',
    'first_glucose (serum)',
    'first_glucose (whole blood)',
    'first_glucose finger stick (range 70-100)',
    'first_hco3 (serum)',
    'first_hematocrit (serum)',
    'first_hemoglobin',
    'first_inr',
    'first_ionized calcium',
    'first_ldh',
    'first_lactic acid',
    'first_magnesium',
    'first_ph (arterial)',
    'first_ph (dipstick)',
    'first_ptt',
    'first_phosphorous',
    'first_platelet count',
    'first_potassium (serum)',
    'first_potassium (whole blood)',
    'first_prothrombin time',
    'first_sodium (serum)',
    'first_specific gravity (urine)',
    'first_tco2 (calc) arterial',
    'first_total bilirubin',
    'first_troponin-t',
    'first_wbc',
    'last_alt',
    'last_ast',
    'last_albumin',
    'last_alkaline phosphate',
    'last_anion gap',
    'last_arterial base excess',
    'last_arterial co2 pressure',
    'last_arterial o2 pressure',
    'last_bun',
    'last_ck (cpk)',
    'last_ck-mb',
    'last_calcium non-ionized',
    'last_chloride (serum)',
    'last_creatinine (serum)',
    'last_differential-basos',
    'last_differential-eos',
    'last_differential-lymphs',
    'last_differential-monos',
    'last_differential-neuts',
    'last_glucose (serum)',
    'last_glucose (whole blood)',
    'last_glucose finger stick (range 70-100)',
    'last_hco3 (serum)',
    'last_hematocrit (serum)',
    'last_hemoglobin',
    'last_inr',
    'last_ionized calcium',
    'last_ldh',
    'last_lactic acid',
    'last_magnesium',
    'last_ph (arterial)',
    'last_ph (dipstick)',
    'last_ptt',
    'last_phosphorous',
    'last_platelet count',
    'last_potassium (serum)',
    'last_potassium (whole blood)',
    'last_prothrombin time',
    'last_sodium (serum)',
    'last_sodium (whole blood)',
    'last_specific gravity (urine)',
    'last_tco2 (calc) arterial',
    'last_total bilirubin',
    'last_troponin-t',
    'last_wbc',
    'first_hr',
    'first_hr alarm - high',
    'first_hr alarm - low',
    'first_nbpd',
    'first_nbpm',
    'first_nbps',
    'first_temperature f',
    'last_hr',
    'last_hr alarm - high',
    'last_hr alarm - low',
    'last_nbpd',
    'last_nbpm',
    'last_nbps',
    'last_temperature f',
    'median_hr',
    'median_hr alarm - high',
    'median_hr alarm - low',
    'median_nbpd',
    'median_nbpm',
    'median_nbps',
    'median_temperature f',
    'min_hr',
    'min_hr alarm - high',
    'min_hr alarm - low',
    'min_nbpd',
    'min_nbpm',
    'min_nbps',
    'min_temperature f',
    'max_hr',
    'max_hr alarm - high',
    'max_hr alarm - low',
    'max_nbpd',
    'max_nbpm',
    'max_nbps',
    'max_temperature f',
    'output',
    'm01',
    'b01',
    'c03',
    'c02']]

In [400]:
data.columns.to_list()

['stay_id',
 'subject_id',
 'hadm_id',
 'icu_day_id',
 'race',
 'gender',
 'age',
 'bmi',
 'height',
 'weight',
 'diastolic_bp',
 'systolic_bp',
 'first_alt',
 'first_ast',
 'first_albumin',
 'first_alkaline phosphate',
 'first_anion gap',
 'first_arterial base excess',
 'first_arterial co2 pressure',
 'first_arterial o2 pressure',
 'first_bun',
 'first_ck (cpk)',
 'first_ck-mb',
 'first_calcium non-ionized',
 'first_chloride (serum)',
 'first_creatinine (serum)',
 'first_differential-basos',
 'first_differential-eos',
 'first_differential-lymphs',
 'first_differential-monos',
 'first_differential-neuts',
 'first_glucose (serum)',
 'first_glucose (whole blood)',
 'first_glucose finger stick (range 70-100)',
 'first_hco3 (serum)',
 'first_hematocrit (serum)',
 'first_hemoglobin',
 'first_inr',
 'first_ionized calcium',
 'first_ldh',
 'first_lactic acid',
 'first_magnesium',
 'first_ph (arterial)',
 'first_ph (dipstick)',
 'first_ptt',
 'first_phosphorous',
 'first_platelet count',
 'fir

In [276]:
for k, v  in data.isna().sum().sort_values(ascending=False).iteritems():
    print(k, v)

last_Sodium (whole blood) 14353
last_Specific Gravity (urine) 14001
first_Specific Gravity (urine) 14001
last_PH (dipstick) 13745
first_PH (dipstick) 13745
first_CK-MB 13714
last_CK-MB 13714
last_CK (CPK) 13708
first_CK (CPK) 13708
first_Potassium (whole blood) 13616
last_Potassium (whole blood) 13616
last_Glucose (whole blood) 13396
first_Glucose (whole blood) 13396
last_Troponin-T 13166
first_Troponin-T 13166
first_LDH 12976
last_LDH 12976
first_Albumin 12849
last_Albumin 12849
first_Differential-Monos 12047
last_Differential-Basos 12047
last_Differential-Eos 12047
last_Differential-Monos 12047
last_Differential-Neuts 12047
first_Differential-Neuts 12047
first_Differential-Basos 12047
first_Differential-Eos 12047
last_Differential-Lymphs 12046
first_Differential-Lymphs 12046
Height 11955
BMI 11004
systolic_bp 10882
diastolic_bp 10882
first_Ionized Calcium 10832
last_Ionized Calcium 10832
Weight 10486
first_Total Bilirubin 10199
last_Total Bilirubin 10199
first_Alkaline Phosphate 1019

In [304]:
data.subject_id = data.subject_id.astype(int)
data.hadm_id = data.hadm_id.astype(int)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [363]:
for col in data.columns:
    print(col)

stay_id
subject_id
hadm_id
hadm_id
icu_day_id
race
gender
age
bmi
height
weight
diastolic_bp
systolic_bp
first_alt
first_ast
first_albumin
first_alkaline phosphate
first_anion gap
first_arterial base excess
first_arterial co2 pressure
first_arterial o2 pressure
first_bun
first_ck (cpk)
first_ck-mb
first_calcium non-ionized
first_chloride (serum)
first_creatinine (serum)
first_differential-basos
first_differential-eos
first_differential-lymphs
first_differential-monos
first_differential-neuts
first_glucose (serum)
first_glucose (whole blood)
first_glucose finger stick (range 70-100)
first_hco3 (serum)
first_hematocrit (serum)
first_hemoglobin
first_inr
first_ionized calcium
first_ldh
first_lactic acid
first_magnesium
first_ph (arterial)
first_ph (dipstick)
first_ptt
first_phosphorous
first_platelet count
first_potassium (serum)
first_potassium (whole blood)
first_prothrombin time
first_sodium (serum)
first_specific gravity (urine)
first_tco2 (calc) arterial
first_total bilirubin
first_t

In [458]:
data.age = data.age.astype(int) 

In [494]:
train_data = data[data.stay_id.isin(train_stays)].drop_duplicates()
test_data = data[data.stay_id.isin(test_stays)].drop_duplicates()
val_data = data[data.stay_id.isin(val_stays)].drop_duplicates()

In [402]:
for col in train_data.columns:
    print(col)

stay_id
subject_id
hadm_id
icu_day_id
race
gender
age
bmi
height
weight
diastolic_bp
systolic_bp
first_alt
first_ast
first_albumin
first_alkaline phosphate
first_anion gap
first_arterial base excess
first_arterial co2 pressure
first_arterial o2 pressure
first_bun
first_ck (cpk)
first_ck-mb
first_calcium non-ionized
first_chloride (serum)
first_creatinine (serum)
first_differential-basos
first_differential-eos
first_differential-lymphs
first_differential-monos
first_differential-neuts
first_glucose (serum)
first_glucose (whole blood)
first_glucose finger stick (range 70-100)
first_hco3 (serum)
first_hematocrit (serum)
first_hemoglobin
first_inr
first_ionized calcium
first_ldh
first_lactic acid
first_magnesium
first_ph (arterial)
first_ph (dipstick)
first_ptt
first_phosphorous
first_platelet count
first_potassium (serum)
first_potassium (whole blood)
first_prothrombin time
first_sodium (serum)
first_specific gravity (urine)
first_tco2 (calc) arterial
first_total bilirubin
first_troponin-

In [510]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(10477, 141)
(1325, 141)
(1307, 141)


In [512]:
train_data[train_data.stay_id.isin(aki_stage_labels.stay_id.unique())]

,stay_id,subject_id,hadm_id,icu_day_id,race,gender,age,bmi,height,weight,...,max_hr alarm - low,max_nbpd,max_nbpm,max_nbps,max_temperature f,output,m01,b01,c03,c02
0,31090461,10002155,28994087.0,0.0,1,1,82,21.40,60.00,109.40,...,50.0,25.00,39.0,77.00,96.9,3260.0,0,1,0,0
2,35479615,10002428,23473524.0,0.0,1,1,81,20.00,59.00,99.00,...,60.0,50.47,63.0,96.86,97.2,2255.0,0,1,0,0
3,33987268,10002428,28662225.0,0.0,1,1,81,20.00,59.00,99.00,...,50.0,19.00,38.0,73.00,98.6,805.0,0,1,0,0
7,38875437,10002428,28662225.0,0.0,1,1,81,20.00,59.00,99.00,...,50.0,30.00,34.0,48.00,98.5,1630.0,0,0,0,0
11,35044219,10002443,21329021.0,0.0,1,2,53,45.90,70.00,320.00,...,40.0,56.00,69.0,86.00,96.7,3450.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18068,31877557,19997293,28847872.0,0.0,1,2,76,32.02,65.56,172.21,...,50.0,25.00,45.0,88.00,97.4,767.0,0,1,0,0
18069,34531437,19997752,29452285.0,0.0,1,1,66,27.60,67.00,176.37,...,45.0,35.00,55.0,109.00,97.2,1540.0,1,0,1,1
18070,32880062,19997843,20277361.0,0.0,5,2,47,32.02,65.56,172.21,...,40.0,56.00,66.0,88.00,98.4,2920.0,0,0,0,0
18071,37403074,19997886,20793010.0,0.0,1,2,72,23.40,67.00,149.69,...,50.0,48.00,61.0,82.00,98.2,1850.0,0,0,0,0


In [513]:
aki_stage_labels[aki_stage_labels.stay_id==31090461]

,subject_id,hadm_id,stay_id,specimen_id,day_id,icu_day_id,icu_12h_window_id,charttime,intime,AKI_1_scr,AKI_2_scr,AKI_3_scr,AKI_1_urine,AKI_2_urine,AKI_3_urine,AKI_1,AKI_2,AKI_3,ANY_AKI,NO_AKI
201204,10002155,28994087,31090461,33825857.0,1,1,2,2130-09-25 01:49:00,2130-09-24 00:50:00,0,0,0,0.0,0.0,0.0,0,0,0,0,1


### Some statistics

In [191]:
data.gender.isna().sum()

1154

In [184]:
print(data.shape[0])
print(data.stay_id.unique().shape[0])

14282
14282


In [182]:
np.sum(((data.gender) == 1).values) / data.shape[0]

0.41632824534378937

In [181]:
np.sum(((data.gender) == 2).values) / data.shape[0]

0.5004901274331326

In [199]:
np.sum(((data.gender) == 1).values) / np.sum(((data.gender) == 2).values)

0.7761256764566805

In [175]:
data[['age']].describe()

,gender,age
count,13094.000000,13094.000000
mean,1.545899,63.575531
std,0.497908,17.314115
min,1.000000,18.000000
25%,1.000000,53.000000
50%,2.000000,65.000000
75%,2.000000,77.000000
max,2.000000,100.000000


In [197]:
data_patients[(data_patients.subject_id.isin(data.subject_id_x))|(data_patients.subject_id.isin(data.subject_id_y))]

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod


## Missing value inputation

In [514]:
def impute_missing(data, columns_to_norm):
    df = data.copy()
    for column in columns_to_norm:
        try:
            df[column] = df[column].fillna(np.round(df[column].mean(), 2))
        except:
            print(column)
            raise

    return df

In [313]:
[*data_vitals_columns, *data_labs_columns, *['output'], *dataset_demo_cols]

['first_HR',
 'first_HR Alarm - High',
 'first_HR Alarm - Low',
 'first_NBPd',
 'first_NBPm',
 'first_NBPs',
 'first_Temperature F',
 'last_HR',
 'last_HR Alarm - High',
 'last_HR Alarm - Low',
 'last_NBPd',
 'last_NBPm',
 'last_NBPs',
 'last_Temperature F',
 'median_HR',
 'median_HR Alarm - High',
 'median_HR Alarm - Low',
 'median_NBPd',
 'median_NBPm',
 'median_NBPs',
 'median_Temperature F',
 'min_HR',
 'min_HR Alarm - High',
 'min_HR Alarm - Low',
 'min_NBPd',
 'min_NBPm',
 'min_NBPs',
 'min_Temperature F',
 'max_HR',
 'max_HR Alarm - High',
 'max_HR Alarm - Low',
 'max_NBPd',
 'max_NBPm',
 'max_NBPs',
 'max_Temperature F',
 'first_ALT',
 'first_AST',
 'first_Albumin',
 'first_Alkaline Phosphate',
 'first_Anion gap',
 'first_Arterial Base Excess',
 'first_Arterial CO2 Pressure',
 'first_Arterial O2 pressure',
 'first_BUN',
 'first_CK (CPK)',
 'first_CK-MB',
 'first_Calcium non-ionized',
 'first_Chloride (serum)',
 'first_Creatinine (serum)',
 'first_Differential-Basos',
 'first_Di

In [497]:
cols_to_impute = data.columns.to_list()[4:]
cols_to_impute

['race',
 'gender',
 'age',
 'bmi',
 'height',
 'weight',
 'diastolic_bp',
 'systolic_bp',
 'first_alt',
 'first_ast',
 'first_albumin',
 'first_alkaline phosphate',
 'first_anion gap',
 'first_arterial base excess',
 'first_arterial co2 pressure',
 'first_arterial o2 pressure',
 'first_bun',
 'first_ck (cpk)',
 'first_ck-mb',
 'first_calcium non-ionized',
 'first_chloride (serum)',
 'first_creatinine (serum)',
 'first_differential-basos',
 'first_differential-eos',
 'first_differential-lymphs',
 'first_differential-monos',
 'first_differential-neuts',
 'first_glucose (serum)',
 'first_glucose (whole blood)',
 'first_glucose finger stick (range 70-100)',
 'first_hco3 (serum)',
 'first_hematocrit (serum)',
 'first_hemoglobin',
 'first_inr',
 'first_ionized calcium',
 'first_ldh',
 'first_lactic acid',
 'first_magnesium',
 'first_ph (arterial)',
 'first_ph (dipstick)',
 'first_ptt',
 'first_phosphorous',
 'first_platelet count',
 'first_potassium (serum)',
 'first_potassium (whole blood)

In [498]:
train_data = impute_missing(train_data, cols_to_impute)
val_data = impute_missing(val_data, cols_to_impute)
test_data = impute_missing(test_data, cols_to_impute)

In [406]:
train_data.isna().sum().sort_values()

stay_id                    0
first_hr alarm - high      0
first_hr                   0
last_wbc                   0
last_troponin-t            0
                        ... 
first_sodium (serum)       0
first_ptt                  0
c02                        0
hadm_id                   18
icu_day_id               147
Length: 141, dtype: int64

In [358]:
train_data.shape

(13263, 144)

In [407]:
for col in train_data.columns:
    print(col)

stay_id
subject_id
hadm_id
icu_day_id
race
gender
age
bmi
height
weight
diastolic_bp
systolic_bp
first_alt
first_ast
first_albumin
first_alkaline phosphate
first_anion gap
first_arterial base excess
first_arterial co2 pressure
first_arterial o2 pressure
first_bun
first_ck (cpk)
first_ck-mb
first_calcium non-ionized
first_chloride (serum)
first_creatinine (serum)
first_differential-basos
first_differential-eos
first_differential-lymphs
first_differential-monos
first_differential-neuts
first_glucose (serum)
first_glucose (whole blood)
first_glucose finger stick (range 70-100)
first_hco3 (serum)
first_hematocrit (serum)
first_hemoglobin
first_inr
first_ionized calcium
first_ldh
first_lactic acid
first_magnesium
first_ph (arterial)
first_ph (dipstick)
first_ptt
first_phosphorous
first_platelet count
first_potassium (serum)
first_potassium (whole blood)
first_prothrombin time
first_sodium (serum)
first_specific gravity (urine)
first_tco2 (calc) arterial
first_total bilirubin
first_troponin-

In [634]:
with open(DF_PATH + 'train_data_icu_multihot.pkl', 'wb') as f:
    pickle.dump(train_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [635]:
with open(DF_PATH + 'val_data_icu_multihot.pkl', 'wb') as f:
    pickle.dump(val_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [636]:
with open(DF_PATH + 'test_data_icu_multihot.pkl', 'wb') as f:
    pickle.dump(test_data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open(DF_PATH + 'train_data_icu_multihot.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open(DF_PATH + 'val_data_icu_multihot.pkl', 'rb') as f:
    val_data = pickle.load(f)

with open(DF_PATH + 'test_data_icu_multihot.pkl', 'rb') as f:
    test_data = pickle.load(f)

# Dataset

In [515]:
train_data.head(1)

,stay_id,subject_id,hadm_id,icu_day_id,race,gender,age,bmi,height,weight,...,max_hr alarm - low,max_nbpd,max_nbpm,max_nbps,max_temperature f,output,m01,b01,c03,c02
0,31090461,10002155,28994087.0,0.0,1,1,82,21.4,60.0,109.4,...,50.0,25.0,39.0,77.0,96.9,3260.0,0,1,0,0


In [463]:
for k, v in train_data.dtypes.iteritems():
    print(k,v)

stay_id int64
subject_id int64
hadm_id float64
icu_day_id float64
race int64
gender int64
age int64
bmi float64
height float64
weight float64
diastolic_bp float64
systolic_bp float64
first_alt float64
first_ast float64
first_albumin float64
first_alkaline phosphate float64
first_anion gap float64
first_arterial base excess float64
first_arterial co2 pressure float64
first_arterial o2 pressure float64
first_bun float64
first_ck (cpk) float64
first_ck-mb float64
first_calcium non-ionized float64
first_chloride (serum) float64
first_creatinine (serum) float64
first_differential-basos float64
first_differential-eos float64
first_differential-lymphs float64
first_differential-monos float64
first_differential-neuts float64
first_glucose (serum) float64
first_glucose (whole blood) float64
first_glucose finger stick (range 70-100) float64
first_hco3 (serum) float64
first_hematocrit (serum) float64
first_hemoglobin float64
first_inr float64
first_ionized calcium float64
first_ldh float64
first_

In [683]:
aki_stage_labels.iloc[15:30, -5:]

,AKI_1,AKI_2,AKI_3,ANY_AKI,NO_AKI
171,0,0,0,0,1
192,0,0,0,0,1
194,0,0,0,0,1
218,0,0,0,0,1
220,0,0,0,0,1
227,0,0,0,0,1
277,0,0,0,0,1
279,0,0,0,0,1
291,0,0,0,0,1
317,0,1,0,1,0


In [5]:
class Dataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels_df = labels[labels.icu_day_id==1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tensor_data = torch.tensor(self.data.values[idx][4:], dtype=torch.double)
        stay_id = self.data.values[idx][0]

        AKI_1_label = (np.sum(self.labels_df.AKI_1.values[idx]) > 0).astype(int)
        AKI_2_label = (np.sum(self.labels_df.AKI_2.values[idx]) > 0).astype(int)
        AKI_3_label = (np.sum(self.labels_df.AKI_3.values[idx]) > 0).astype(int)
        NO_AKI_label = (np.sum(self.labels_df.NO_AKI.values[idx]) > 0).astype(int)
        tensor_labels = torch.tensor([AKI_1_label, AKI_2_label, AKI_3_label, NO_AKI_label], dtype=torch.double)

        return tensor_data, tensor_labels
        

In [688]:
aki_stage_labels.iloc[0].AKI_1.values

AttributeError: 'numpy.int64' object has no attribute 'values'

In [518]:
labels = aki_stage_labels.copy()
labels = labels[labels.icu_day_id==1]

In [6]:
train_dataset = Dataset(train_data, aki_stage_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [7]:
sample, label = next(iter(train_loader))

In [709]:
label

tensor([[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]], dtype=torch.float64)

In [552]:
sample.size()

torch.Size([8, 137])

In [641]:
i = 0
for tensor_data, tensor_labels  in train_loader:
    print(tensor_data.size(),tensor_labels.size()  )
    i += 1

RuntimeError: stack expects each tensor to be equal size, but got [2, 4] at entry 0 and [1, 4] at entry 2

In [655]:
tensor_data, tensor_labels  = train_dataset[2]
tensor_labels

tensor([[0, 1, 0, 0],
        [1, 0, 0, 0]], dtype=torch.uint8)

In [660]:
i = 0
for i in range(len(train_dataset)):
    tensor_data, tensor_labels  = train_dataset[i]
    if i == 0:
        size = tensor_labels.size()
    if tensor_labels.size() != size:
        print(i)
        break

In [8]:
class EHR_MODEL(nn.Module):
    def __init__(self,  drop=0.6, hidden_size=128):
        super(EHR_MODEL, self).__init__()

        self.p = drop
        self.input_size = 137
        self.H = hidden_size
        # layers of the network
        self.lstm = nn.LSTM(input_size=self.input_size,
                              hidden_size=self.H,
                              num_layers=1,
                              batch_first=True,
                              bidirectional=True)

        self.fc_1 = nn.Linear(2 * self.H , 512)
        self.fc_2 = nn.Linear(512, 4)
        self.drop = nn.Dropout(p=self.p)

    def forward(self, data):
        # data = data.to(torch.double) 
        out_lstm, _ = self.lstm(data.unsqueeze(1))
        out_lstm = out_lstm.reshape(out_lstm.size(0), out_lstm.size(1)*out_lstm.size(2))

        output = self.fc_1(out_lstm)
        output = self.drop(output)
        output = self.fc_2(output)
        output = torch.squeeze(output, 1)

        return output


In [9]:
model = EHR_MODEL().to(torch.double)

In [620]:
print(sample.size())
print(sample.unsqueeze(1).size())

torch.Size([8, 137])
torch.Size([8, 1, 137])


In [615]:
inputs = [torch.randn(1, 3) for _ in range(5)]
inputs[0].view(1,1,-1)

tensor([[[1.0015, 1.1146, 0.8885]]])

In [622]:
out_lstm, _ = model.lstm(sample.unsqueeze(1))

In [623]:
out_lstm.size()

torch.Size([8, 1, 256])

In [10]:
sample, label = next(iter(train_loader))

In [12]:
output = model(sample)

In [13]:
stacked_probs = nn.Softmax()(output)
stacked_probs = stacked_probs.cpu().detach().numpy()

/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [14]:
y_pred = np.argmax(stacked_probs, axis=1)
y_true = np.argmax(label, axis=1)


In [17]:
confusion_matrix(y_true, y_pred)

array([[1, 0, 0],
       [0, 0, 1],
       [3, 2, 1]])

In [18]:
print(classification_report(y_true, y_pred, zero_division=0, output_dict=False))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         1
           3       0.50      0.17      0.25         6

    accuracy                           0.25         8
   macro avg       0.25      0.39      0.22         8
weighted avg       0.41      0.25      0.24         8

